# Neuro-Fuzzy System for Customer 360 Profiling and Churn Prediction

A 360-degree enriched customer profile is key to derive actionable insights and smarter data-driven decisions.

Customer profile enrichment can be applied across multiple business use cases:

- Design targeted sales events.
- Personalize promotions and offers
- Proactively reduce customer churn
- Predict customer buying behavior
- Forecast demands for better pricing and inventory
- Gain actionable insights for better sales and opportunity leads

Some of the benefits of customer profile enrichment includes:

- A 360-degree representation of the customer.
- Holistic view of your ideal customers’ purchasing behaviors and patterns.
- Allows you to locate, understand and better connect to your ideal customers.

### Example for car sharing fleet management and maintenance tickets.

In [ ]:
# useful packages
import numpy as np
import matplotlib.pyplot as plt
from obs import *
import csv
import io
import time
# imports for fuzzy inference system in churn prediction
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

In [ ]:
# load first dataset
dataPairIdentifier1 = 'customer_nrActiveCarsInFleet_nrServiceComplaints.csv';

# load the second dataset 
dataPairIdentifier2 = 'customer_nrActiveCarsInFleet_nrMaintenanceTickets.csv';

# load the third dataset 
dataPairIdentifier3 = 'customer_nrServiceComplaints_nrMaintenanceTickets.csv';

In [ ]:
# compute the value of the logistic function for single neuron dynamics
# given the slope, m, and the shift, s
def compute_s(x, m ,s):
    y = np.zeros((len(x), 1))
    for idx in range(len(x)):
        y[idx, 0] = 1/(1 + np.exp(-m*(x[idx, 0] - s)))
    return y

In [ ]:
# compute topological distance between activations in each of the neurons
def compute_d(N, s):
    y = np.zeros((N, N))
    for idx in range(N):
        for jdx in range(N):
            y[idx, jdx] = np.exp(-0.5*(np.min([np.abs(idx-jdx), N - np.abs(idx-jdx)])/s) ** 2)
    return y

In [ ]:
# create the learning network composed of N_POP populations of
# N_NEURONS neurons and init each struct weight and activity matrices
def create_init_network(N_POP, N_NEURONS, GAMMA, SIGMA, DELTA, MAX_INIT_RANGE, TARGET_VAL_ACT):
    wta_profile = GAMMA * compute_d(N_NEURONS, SIGMA) - DELTA;
    wext = np.random.rand(N_NEURONS, N_NEURONS)*MAX_INIT_RANGE;
    populations = {}
    for pop_idx in range(N_POP):
        populations[pop_idx] = {'idx': pop_idx,
                                'lsize': N_NEURONS,
                                'Wint': wta_profile,
                                'Wext': wext / np.sum(wext),
                                'a': np.zeros((N_NEURONS, 1))*TARGET_VAL_ACT,
                                'h': np.zeros((N_NEURONS, 1))*TARGET_VAL_ACT
                               }
    return populations

In [ ]:
# function to generate the population encoded variable as input for the net
# here we also need to encode variables which are in both +/- ranges
# we need to take into accound the encoding for the tuning curves
# distribution
def population_encoder(x, range_input, N):
    sig = .1  # Standard deviation
    K = 1; # max firing rate (Hz) (ignore - not modeling nurophysiology here :)
    # pattern of activity, or output tuning curve between [-range, range]
    R = np.zeros((N, 1))
    # calculate output 
    for j in range(N):  # for each neuron in the population
        R[j, 0] = K*np.exp( -(x - (-range_input+(j)*(range_input/((N)/2))))**2 / (2*sig**2))
    return R

In [ ]:
# plot a heatmap 
def print_heatmap(arr):
    plt.figure()
    plt.imshow(arr)
    plt.show()

# Fuzzy Inference Engine
To describe the business logic in customer profiling and churning.

In [ ]:
# load data
dataActiveCarsInFleet = trainDataChurn1
dataMaintenanceTickets = trainDataChurn2

# random seed init
np.random.seed(0)
# build the fuzzy inference variables
# inputs
nrActiveCarsInFleet = ctrl.Antecedent(universe=np.arange(min(dataActiveCarsInFleet), max(dataActiveCarsInFleet), 1), label='nrActiveCarsInFleet')
nrMaintenanceTickets = ctrl.Antecedent(universe=np.arange(min(dataMaintenanceTickets), max(dataMaintenanceTickets), 1), label='nrMaintenanceTickets')
# output
churnProbability = ctrl.Consequent(universe=np.arange(0, 101, 1), label='churnProbability')

# parametrize the variables membership functions
# inputs
nrActiveCarsInFleet.automf(number=3, variable_type='number', names=['low', 'medium', 'high'])
nrMaintenanceTickets.automf(number=3, variable_type='number', names=['low', 'medium', 'high'])
# output
churnProbability['low'] = fuzz.trimf(x=churnProbability.universe, abc=[0, 0, 50])
churnProbability['medium'] = fuzz.trimf(x=churnProbability.universe, abc=[0, 50, 100])
churnProbability['high'] = fuzz.trimf(x=churnProbability.universe, abc=[50, 100, 100])


# add the BI rules for churning
rule1 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['low'] & nrMaintenanceTickets['low'],
    consequent=churnProbability['low']
)

rule2 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['low'] & nrMaintenanceTickets['medium'],
    consequent=churnProbability['medium']
)

rule3 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['medium'] & nrMaintenanceTickets['low'],
    consequent=churnProbability['low']
)

rule4 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['medium'] & nrMaintenanceTickets['medium'],
    consequent=churnProbability['medium']
)

rule5 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['high'] & nrMaintenanceTickets['low'],
    consequent=churnProbability['low']
)

rule6 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['low'] & nrMaintenanceTickets['high'],
    consequent=churnProbability['high']
)


rule7 = ctrl.Rule(
    antecedent=nrActiveCarsInFleet['medium'] & nrMaintenanceTickets['high'],
    consequent=churnProbability['high']
)


# build the inference engine
churnProbability_ctrl = ctrl.ControlSystem(rules=[rule1, rule2, rule3, rule4, rule5, rule6, rule7])

# build 
churnProbability = ctrl.ControlSystemSimulation(control_system=churnProbability_ctrl)
churnProbability.reset()

### Neural network to learn pairwise relations between the features of the customer. 
The final 360 profile is based on the extracted relations.

In [ ]:
## INIT SIMULATION 
# enables dynamic visualization on network runtime
DYN_VISUAL      = 1
# number of populations in the network
N_POP           = 2
# number of neurons in each population
N_NEURONS       = 200
# max range value @ init for weights and activities in the population
MAX_INIT_RANGE  = 1
# WTA circuit settling threshold
EPSILON         = 1e-3
## INIT NETWORK DYNAMICS
# epoch iterator in outer loop (HL, HAR)
t       = 1;
# network iterator in inner loop (WTA)
tau     = 1;
# constants for WTA circuit (convolution based WTA), these will provide a
# profile peaked at ~ TARGET_VAL_ACT
DELTA   = -0.005                   # displacement of the convolutional kernel (neighborhood)
SIGMA   = 5.0                      # standard deviation in the exponential update rule
SL      = 4.5                      # scaling factor of neighborhood kernel
GAMMA   = SL/(SIGMA*np.sqrt(2*np.pi))    # convolution scaling factor
# constants for Hebbian linkage
ALPHA_L = 1.0*1e-2                 # Hebbian learning rate
ALPHA_D = 1.0*1e-2                 # Hebbian decay factor ALPHA_D >> ALPHA_L
# constants for HAR
C       = 0.005                    # scaling factor in homeostatic activity regulation
TARGET_VAL_ACT  = 0.4              # amplitude target for HAR
A_TARGET        = TARGET_VAL_ACT*np.ones((N_NEURONS, 1)) # HAR target activity vector
# constants for neural units in neural populations
M       = 1; # slope in logistic function @ neuron level
S       = 10.0; # shift in logistic function @ neuron level
# activity change weight (history vs. incoming knowledge)
ETA     = 0.25;

## SENSORY DATA SETUP 

# prepare first dataset
# bind sensory data
sensory_data1 = trainData1;

# get data len
DATASET_LEN     = len(sensory_data1[:, 0])

# prepare second dataset
# bind sensory data
sensory_data2 = trainData2;

# prepare third dataset
# bind sensory data
sensory_data3 = trainData3;

# churn history
churn_history = np.zeros((DATASET_LEN, 1))

## CREATE NETWORK AND INITIALIZE

# create network for the first dataset
# create a network given the simulation constants
populations1 = create_init_network(N_POP, N_NEURONS, GAMMA, SIGMA, DELTA, MAX_INIT_RANGE, TARGET_VAL_ACT)
# buffers for changes in activity in WTA loop
act1 = np.zeros((N_NEURONS, N_POP))*MAX_INIT_RANGE
old_act1 = np.zeros((N_NEURONS, N_POP))*MAX_INIT_RANGE
# buffers for running average of population activities in HAR loop
old_avg1 = np.zeros((N_POP, N_NEURONS))
cur_avg1 = np.zeros((N_POP, N_NEURONS))
# the new rate values
delta_a0_1 = np.zeros((N_NEURONS, 1))
delta_a1_1 = np.zeros((N_NEURONS, 1))
tau1 = tau

# create network for the second dataset
# create a network given the simulation constants
populations2 = create_init_network(N_POP, N_NEURONS, GAMMA, SIGMA, DELTA, MAX_INIT_RANGE, TARGET_VAL_ACT)
# buffers for changes in activity in WTA loop
act2 = np.zeros((N_NEURONS, N_POP))*MAX_INIT_RANGE
old_act2 = np.zeros((N_NEURONS, N_POP))*MAX_INIT_RANGE
# buffers for running average of population activities in HAR loop
old_avg2 = np.zeros((N_POP, N_NEURONS))
cur_avg2 = np.zeros((N_POP, N_NEURONS))
# the new rate values
delta_a0_2 = np.zeros((N_NEURONS, 1))
delta_a1_2 = np.zeros((N_NEURONS, 1))
tau2 = tau

# create network for the third dataset
# create a network given the simulation constants
populations3 = create_init_network(N_POP, N_NEURONS, GAMMA, SIGMA, DELTA, MAX_INIT_RANGE, TARGET_VAL_ACT)
# buffers for changes in activity in WTA loop
act3 = np.zeros((N_NEURONS, N_POP))*MAX_INIT_RANGE
old_act3 = np.zeros((N_NEURONS, N_POP))*MAX_INIT_RANGE
# buffers for running average of population activities in HAR loop
old_avg3 = np.zeros((N_POP, N_NEURONS))
cur_avg3 = np.zeros((N_POP, N_NEURONS))
# the new rate values
delta_a0_3 = np.zeros((N_NEURONS, 1))
delta_a1_3 = np.zeros((N_NEURONS, 1))
tau3 = tau

## NETWORK SIMULATION LOOP
# # present each entry in the dataset for MAX_EPOCHS epochs to train the net
for didx in range(DATASET_LEN):
      
    # loop for the first dataset
    # pick a new sample from the dataset and feed it to the input (noiseless input)
    # population in the network (in this case X -> A -> | <- B <- Y)
    X1 = population_encoder(sensory_data1[didx, 0], np.max(sensory_data1[:, 0]),  N_NEURONS)
    Y1 = population_encoder(sensory_data1[didx, 1], np.max(sensory_data1[:, 1]),  N_NEURONS)
    # normalize input such that the activity in all units sums to 1.0
    X1 /= np.sum(X1)
    Y1 /= np.sum(Y1)
    # clamp input to neural populations
    populations1[0]['a'] = X1
    populations1[1]['a'] = Y1
    # given the input sample wait for WTA circuit to settle and then
    # perform a learning step of Hebbian learning and HAR
    while True:
        # compute changes in activity
        delta_a0_1 = compute_s(populations1[0]['h'] + np.matmul(populations1[0]['Wext'], populations1[1]['a']) + 
                             np.matmul(populations1[0]['Wint'],populations1[0]['a']), 
                             M, 
                             S)
        delta_a1_1 = compute_s(populations1[1]['h'] + np.matmul(populations1[1]['Wext'], populations1[0]['a']) + 
                             np.matmul(populations1[1]['Wint'], populations1[1]['a']), 
                             M, 
                             S)
        # update the activities of each population
        populations1[0]['a'] = (1-ETA)*populations1[0]['a'] + ETA*delta_a0_1
        populations1[1]['a'] = (1-ETA)*populations1[1]['a'] + ETA*delta_a1_1
        # current activation values holder
        for pop_idx in range(N_POP):
            act1[:, pop_idx] = populations1[pop_idx]['a'].reshape(-1)
        # check if activity has settled in the WTA loop
        q1 = (np.sum(np.sum(np.abs(act1 - old_act1)))/(N_POP*N_NEURONS))
        if q1 <= EPSILON:
            tau1 = 1
            break
        # update history of activities
        old_act1 = act1
        # increment time step in WTA loop
        tau1 += 1
    # update Hebbian linkage between the populations (decaying Hebbian rule)
    populations1[0]['Wext'] = (1-ALPHA_D)*populations1[0]['Wext'] + ALPHA_L*np.matmul(populations1[0]['a'],populations1[1]['a'].transpose())
    populations1[1]['Wext'] = (1-ALPHA_D)*populations1[1]['Wext'] + ALPHA_L*np.matmul(populations1[1]['a'],populations1[0]['a'].transpose())
    # compute the inverse time for exponential averaging of HAR activity
    omegat1 = 0.002 + 0.998/(t+2)
    # for each population in the network
    for pop_idx in range(N_POP):
        # update Homeostatic Activity Regulation terms
        # compute exponential average of each population at current step
        cur_avg1[pop_idx, :] = (1-omegat1)*old_avg1[pop_idx, :] + omegat1*populations1[pop_idx]['a'].transpose()
        # update homeostatic activity terms given current and target act.
        populations1[pop_idx]['h'] = populations1[pop_idx]['h'] + C*(TARGET_VAL_ACT - cur_avg1[pop_idx, :].transpose())
    # update averging history
    old_avg1 = cur_avg1
    
    # loop for the second dataset
    # pick a new sample from the dataset and feed it to the input (noiseless input)
    # population in the network (in this case X -> A -> | <- B <- Y)
    X2 = population_encoder(sensory_data2[didx, 0], np.max(sensory_data2[:, 0]),  N_NEURONS)
    Y2 = population_encoder(sensory_data2[didx, 1], np.max(sensory_data2[:, 1]),  N_NEURONS)
    # normalize input such that the activity in all units sums to 1.0
    X2 /= np.sum(X2)
    Y2 /= np.sum(Y2)
    # clamp input to neural populations
    populations2[0]['a'] = X2
    populations2[1]['a'] = Y2
    # given the input sample wait for WTA circuit to settle and then
    # perform a learning step of Hebbian learning and HAR
    while True:
        # compute changes in activity
        delta_a0_2 = compute_s(populations2[0]['h'] + np.matmul(populations2[0]['Wext'], populations2[1]['a']) + 
                             np.matmul(populations2[0]['Wint'],populations2[0]['a']), 
                             M, 
                             S)
        delta_a1_2 = compute_s(populations2[1]['h'] + np.matmul(populations2[1]['Wext'], populations2[0]['a']) + 
                             np.matmul(populations2[1]['Wint'], populations2[1]['a']), 
                             M, 
                             S)
        # update the activities of each population
        populations2[0]['a'] = (1-ETA)*populations2[0]['a'] + ETA*delta_a0_2
        populations2[1]['a'] = (1-ETA)*populations2[1]['a'] + ETA*delta_a1_2
        # current activation values holder
        for pop_idx in range(N_POP):
            act2[:, pop_idx] = populations2[pop_idx]['a'].reshape(-1)
        # check if activity has settled in the WTA loop
        q2 = (np.sum(np.sum(np.abs(act2 - old_act2)))/(N_POP*N_NEURONS))
        if q2 <= EPSILON:
            tau2 = 1
            break
        # update history of activities
        old_act2 = act2
        # increment time step in WTA loop
        tau2 += 1
    # update Hebbian linkage between the populations (decaying Hebbian rule)
    populations2[0]['Wext'] = (1-ALPHA_D)*populations2[0]['Wext'] + ALPHA_L*np.matmul(populations2[0]['a'],populations2[1]['a'].transpose())
    populations2[1]['Wext'] = (1-ALPHA_D)*populations2[1]['Wext'] + ALPHA_L*np.matmul(populations2[1]['a'],populations2[0]['a'].transpose())
    # compute the inverse time for exponential averaging of HAR activity
    omegat2 = 0.002 + 0.998/(t+2)
    # for each population in the network
    for pop_idx in range(N_POP):
        # update Homeostatic Activity Regulation terms
        # compute exponential average of each population at current step
        cur_avg2[pop_idx, :] = (1-omegat2)*old_avg2[pop_idx, :] + omegat2*populations2[pop_idx]['a'].transpose()
        # update homeostatic activity terms given current and target act.
        populations2[pop_idx]['h'] = populations2[pop_idx]['h'] + C*(TARGET_VAL_ACT - cur_avg2[pop_idx, :].transpose())
    # update averging history
    old_avg2 = cur_avg2
    
    # loop for the third dataset
    # pick a new sample from the dataset and feed it to the input (noiseless input)
    # population in the network (in this case X -> A -> | <- B <- Y)
    X3 = population_encoder(sensory_data3[didx, 0], np.max(sensory_data3[:, 0]),  N_NEURONS)
    Y3 = population_encoder(sensory_data3[didx, 1], np.max(sensory_data3[:, 1]),  N_NEURONS)
    # normalize input such that the activity in all units sums to 1.0
    X3 /= np.sum(X3)
    Y3 /= np.sum(Y3)
    # clamp input to neural populations
    populations3[0]['a'] = X3
    populations3[1]['a'] = Y3
    # given the input sample wait for WTA circuit to settle and then
    # perform a learning step of Hebbian learning and HAR
    while True:
        # compute changes in activity
        delta_a0_3 = compute_s(populations3[0]['h'] + np.matmul(populations3[0]['Wext'], populations3[1]['a']) + 
                             np.matmul(populations3[0]['Wint'],populations3[0]['a']), 
                             M, 
                             S)
        delta_a1_3 = compute_s(populations3[1]['h'] + np.matmul(populations3[1]['Wext'], populations3[0]['a']) + 
                             np.matmul(populations3[1]['Wint'], populations3[1]['a']), 
                             M, 
                             S)
        # update the activities of each population
        populations3[0]['a'] = (1-ETA)*populations3[0]['a'] + ETA*delta_a0_3
        populations3[1]['a'] = (1-ETA)*populations3[1]['a'] + ETA*delta_a1_3
        # current activation values holder
        for pop_idx in range(N_POP):
            act3[:, pop_idx] = populations3[pop_idx]['a'].reshape(-1)
        # check if activity has settled in the WTA loop
        q3 = (np.sum(np.sum(np.abs(act3 - old_act3)))/(N_POP*N_NEURONS))
        if q3 <= EPSILON:
            tau3 = 1
            break
        # update history of activities
        old_act3 = act3
        # increment time step in WTA loop
        tau3 += 1
    # update Hebbian linkage between the populations (decaying Hebbian rule)
    populations3[0]['Wext'] = (1-ALPHA_D)*populations3[0]['Wext'] + ALPHA_L*np.matmul(populations3[0]['a'],populations3[1]['a'].transpose())
    populations3[1]['Wext'] = (1-ALPHA_D)*populations3[1]['Wext'] + ALPHA_L*np.matmul(populations3[1]['a'],populations3[0]['a'].transpose())
    # compute the inverse time for exponential averaging of HAR activity
    omegat3 = 0.002 + 0.998/(t+2)
    # for each population in the network
    for pop_idx in range(N_POP):
        # update Homeostatic Activity Regulation terms
        # compute exponential average of each population at current step
        cur_avg3[pop_idx, :] = (1-omegat3)*old_avg3[pop_idx, :] + omegat3*populations3[pop_idx]['a'].transpose()
        # update homeostatic activity terms given current and target act.
        populations3[pop_idx]['h'] = populations3[pop_idx]['h'] + C*(TARGET_VAL_ACT - cur_avg3[pop_idx, :].transpose())
    # update averging history
    old_avg3 = cur_avg3
    
    # increment timestep for HL and HAR loop
    t = t + 1
    
    # fuzzy inference for churn probability calculation
    churnProbability.inputs({
        'nrActiveCarsInFleet': dataActiveCarsInFleet[didx],
        'nrMaintenanceTickets': dataMaintenanceTickets[didx]
    })
    churnProbability.compute()
    churn_history[didx] = churnProbability.output['churnProbability']
    
    # give time to write to update DLV
    time.sleep(2)
    # write inputs
    write_in(didx, np.matrix(sensory_data1[0:didx, :]), dataPairIdentifier1)
    write_in(didx, np.matrix(sensory_data2[0:didx, :]), dataPairIdentifier2)
    write_in(didx, np.matrix(sensory_data3[0:didx, :]), dataPairIdentifier3)
    # write results
    write_out(populations1[1]['Wext'], dataPairIdentifier1)
    write_out(populations2[1]['Wext'], dataPairIdentifier2)
    write_out(populations3[1]['Wext'], dataPairIdentifier3)
    # write churn probability
    write_churn(didx, churn_history[0:didx])

In [ ]:
# print to verify the learnt relations
print_heatmap(populations1[1]['Wext']); print_heatmap(populations2[1]['Wext']) ; print_heatmap(populations3[1]['Wext']);